# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [4]:
X = boston_features.loc[:,["B", "DIS"]]
y = boston_features["LSTAT"]

In [5]:
X.head()

,B,DIS
0,1.000000,0.542096
1,1.000000,0.623954
2,0.989737,0.623954
3,0.994276,0.707895
4,1.000000,0.707895


In [6]:
y.head()

0   -1.275260
1   -0.263711
2   -1.627858
3   -2.153192
4   -1.162114
Name: LSTAT, dtype: float64

## Train test split

Perform a train-test-split with a test set of 0.20.

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
X_train.head()

,B,DIS
347,0.988729,0.852042
282,0.949997,0.644222
288,1.000000,0.787167
468,0.928993,0.398453
277,0.991301,0.615013


Fit the model and apply the model to the make test set predictions

In [11]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

Calculate the residuals and the mean squared error

In [13]:
predictions = reg.predict(X_test)

residuals = predictions - y_test

mse = (residuals**2).mean()

In [17]:
mse

0.5226882874809512

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [125]:
def kfolds(data, k):
    # Force data as pandas dataframe
    shuffled_data = pd.DataFrame(data)
    # add 1 to fold size to account for leftovers
    folds = []
    shuffled_data = shuffled_data.sample(frac=1).reset_index(drop=True)
    size_of_fold_data = len(data) // k
    num_remaining_data_points = len(data) % k
    for k_index in range(k):
        if k_index < num_remaining_data_points:
            current_index = k_index*size_of_fold_data+k_index
            fold_data = shuffled_data.iloc[current_index:current_index+size_of_fold_data+1,:]
        else:
            current_index = k_index*size_of_fold_data+num_remaining_data_points
            fold_data = shuffled_data.iloc[current_index:current_index+size_of_fold_data,:]
        folds.append(fold_data)
    return folds

### Apply it to the Boston Housing Data

In [126]:
folds = kfolds(boston_features, 5)

In [ ]:
# Make sure to concatenate the data again

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [130]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    fold_data = folds[n]
    X = fold_data.loc[:,["B", "DIS"]]
    y = fold_data["LSTAT"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    # Fit a linear regression model
    reg = LinearRegression().fit(X_train, y_train)
    
    #Evaluate Train and Test Errors
    predictions_train = reg.predict(X_train)
    residuals_train = predictions_train - y_train
    mse_train = (residuals_train**2).mean()
    
    predictions_test = reg.predict(X_test)
    residuals_test = predictions_test - y_test
    mse_test = (residuals_test**2).mean()
    
    train_errs.append(mse_train)
    test_errs.append(mse_test)

print(train_errs)
print(test_errs)

[0.5200915541131916, 0.43369998499497225, 0.6687664580101057, 0.77460080319797, 0.9033703603245291]
[0.890905042069521, 0.8099135755523893, 0.48365229070786786, 1.1538209009935703, 0.41364198460949925]


In [131]:
np.mean(train_errs)

0.6601058321281538

In [132]:
np.mean(test_errs)

0.7503867587865696

## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [144]:
X = boston_features.loc[:,["B", "DIS"]]
y = boston_features["LSTAT"]

In [150]:
from sklearn.model_selection import KFold
scores = []
cv = KFold(n_splits=5, random_state=42, shuffle=False)
for train_index, test_index in cv.split(X):
#     print("Train Index: ", train_index, "\n")
#     print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = X.iloc[train_index,:], X.iloc[test_index,:], y[train_index], y[test_index];
    reg = LinearRegression().fit(X_train, y_train)
    #scores.append(reg.score(X_test, y_test))
    predictions_test = reg.predict(X_test)
    residuals_test = predictions_test - y_test
    mse_test = (residuals_test**2).mean()
    scores.append(mse_test)
print(scores)

[0.8383419755062808, 0.8333416856229062, 1.0211569761877635, 0.6474676378492472, 0.3428288514219758]


In [166]:
from sklearn.model_selection import KFold
scores = []
reg = LinearRegression().fit(X, y)
cv = KFold(n_splits=5, random_state=None, shuffle=True)
for train_index, test_index in cv.split(X):
#     print("Train Index: ", train_index, "\n")
#     print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = X.iloc[train_index,:], X.iloc[test_index,:], y[train_index], y[test_index];
    reg.fit(X_train, y_train)
    #scores.append(reg.score(X_test, y_test))
    predictions_test = reg.predict(X_test)
    residuals_test = predictions_test - y_test
    mse_test = (residuals_test**2).mean()
    scores.append(mse_test)
print(scores)

[0.5701055350627794, 0.8006910816053368, 0.4822916782228979, 0.8648617726998964, 0.7989626093435642]


Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

mean of the 5-fold cross-validation

In [167]:
np.mean(scores)

0.7033825353868949

original test-train-split case 

In [168]:
mse

0.5226882874809512

The mean of the 5-fold cross-validation MSEs is higher than the MSE from the single split.

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!